#### Setup

In [2]:
import json
import logging
import sys
from pathlib import Path
import ipytest
import os
import sagemaker
from sagemaker.workflow.pipeline_context import LocalPipelineSession, PipelineSession
import boto3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from scripts.config import Configuration

CODE_FOLDER = Path("code")
sys.path.extend([f"./{CODE_FOLDER}"])


DATA_FILEPATH = "data/penguins.csv"

ipytest.autoconfig(raise_on_error=True)

logging.getLogger("sagemaker.config").setLevel(logging.ERROR)


In [5]:
LOCAL_MODE = True
config_instance = Configuration(LOCAL_MODE)

In [7]:
bucket = config_instance.bucket
role = config_instance.role
config = config_instance.config
config

{'session': <sagemaker.workflow.pipeline_context.LocalPipelineSession at 0x146ea7af0>,
 'instance_type': 'local',
 'image': 'sagemaker-tensorflow-toolkit-local',
 'framework_version': '2.12',
 'py_version': 'py310'}

In [8]:
S3_LOCATION = f"s3://{bucket}/penguins"

sagemaker_session = sagemaker.session.Session()
sagemaker_client = boto3.client("sagemaker")
iam_client = boto3.client("iam")
region = boto3.Session().region_name